In [53]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

processed_dir = "../processed/"

dataset = pd.read_csv(processed_dir + "rssi_processed.csv", index_col=0)
averages = dataset.mean(axis=0)
missing = dataset.isna().mean() * 100
print("Averages (RSSI Values)\n",averages)
print("\nMissed Scans (%)\n",missing)

Averages (RSSI Values)
 0.5m   -73.348000
1.0m   -81.084337
1.5m   -85.493976
2.0m   -90.417671
2.5m   -92.240964
dtype: float64

Missed Scans (%)
 0.5m    0.0
1.0m    0.4
1.5m    0.4
2.0m    0.4
2.5m    0.4
dtype: float64


In [ ]:
plt.figure(figsize=(10, 6))
# sns.swarmplot(data=dataset, alpha=0.5, split=True)
sns.violinplot(data=dataset, orient="v", alpha=1.0).set_title("Violinplot of RSSI at Different Distances")
plt.xlabel("Distance (m)")
plt.ylabel("RSSI Value (Higher Values Stronger)")

In [ ]:
plt.figure(figsize=(12, 10))
melted = dataset.melt(var_name="distance")
g = sns.FacetGrid(data=melted, col="distance")
g.map(sns.histplot, "value", bins=10)

plt.subplots_adjust(top=0.8)
plt.subplots_adjust(bottom=0)
g.fig.suptitle("Histograms at Each Distance")
g.set_xlabels("RSSI Value (Higher Values Stronger)")
g.set_ylabels("Occurrences of RSSI Values")

In [ ]:
plt.figure(figsize=(10, 6))
sns.kdeplot(data=dataset, fill=True).set_title("Density Estimation of RSSI Values at Different Distances")

plt.xlabel("RSSI Value (Higher Values Stronger)")
plt.ylabel("Density Estimation")

In [126]:
def createEstDistance(mPower, avg, environment):
    #print(10 ** ( (mPower["Measured Power"] - avg) / (10 * environment) ))
    return 0.5 - 10 ** ( (mPower - avg) / (10 * environment) )

def createMeasuredPowerFrame(data, averages):
    env = 2
    powerVals = pd.Series(np.linspace(-100, -50, 51))
    powerVals.name = "Measured Power"
    vals=[powerVals]
    for avg in averages.iteritems():
        series = powerVals.apply(lambda val: createEstDistance(val, avg[1], env) )
        series.name=avg[0]
        vals.append(series)
    return pd.concat(vals, axis=1).set_index("Measured Power")

measuredPowers = createMeasuredPowerFrame(dataset, averages)